In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2022-11-29 08:38:22.165110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 08:38:22.165123: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.8.0-rc1


MNIST dataset for example

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

## Define model

In [5]:
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model

model = create_model()

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

 The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both during and at the end of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training.

In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Epoch 1/10
 1/32 [..............................] - ETA: 7s - loss: 2.3345 - sparse_categorical_accuracy: 0.1562
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 1.1513 - sparse_categorical_accuracy: 0.6740 - val_loss: 0.7679 - val_sparse_categorical_accuracy: 0.7540
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4152 - sparse_categorical_accuracy: 0.8780
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.4152 - sparse_categorical_accuracy: 0.8780 - val_loss: 0.5206 - val_sparse_categorical_accuracy: 0.8390
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2905 - sparse_categorical_accuracy: 0.9250
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2905 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4640 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
 1/32 [..........

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch.

In [7]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [8]:
# Create a basic model instance
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 [==============================] - 0s 666us/step - loss: 2.3961 - sparse_categorical_accuracy: 0.0640
Untrained model, accuracy:  6.40%


In [9]:
# Load the weights
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print('Restores model, accuracy: {:5.2f}%'.format(100*acc))

32/32 [==============================] - 0s 627us/step - loss: 0.4061 - sparse_categorical_accuracy: 0.8740
Restores model, accuracy: 87.40%


## Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs.

In [10]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                verbose=1,
                                                save_weights_only=True,
                                                save_freq=5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
         train_labels,
         epochs=50,
         batch_size=batch_size,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=1)

Epoch 1/50
32/32 [==============================] - 0s 5ms/step - loss: 1.1542 - sparse_categorical_accuracy: 0.6700 - val_loss: 0.7444 - val_sparse_categorical_accuracy: 0.7710
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.4357 - sparse_categorical_accuracy: 0.8700 - val_loss: 0.5345 - val_sparse_categorical_accuracy: 0.8270
Epoch 3/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2923 - sparse_categorical_accuracy: 0.9220 - val_loss: 0.4559 - val_sparse_categorical_accuracy: 0.8600
Epoch 4/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2216 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.4354 - val_sparse_categorical_accuracy: 0.8680
Epoch 5/50
 1/32 [..............................] - ETA: 0s - loss: 0.1241 - sparse_categorical_accuracy: 0.9688
Epoch 5: saving model to training_2/cp-0005.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.1565 - sparse_categorical_accuracy: 0.9640 - val_loss: 0.4425 -

Epoch 36/50
32/32 [==============================] - 0s 3ms/step - loss: 0.0023 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4745 - val_sparse_categorical_accuracy: 0.8750
Epoch 37/50
32/32 [==============================] - 0s 3ms/step - loss: 0.0018 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4789 - val_sparse_categorical_accuracy: 0.8730
Epoch 38/50
32/32 [==============================] - 0s 3ms/step - loss: 0.0022 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4841 - val_sparse_categorical_accuracy: 0.8720
Epoch 39/50
32/32 [==============================] - 0s 2ms/step - loss: 0.0020 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4837 - val_sparse_categorical_accuracy: 0.8780
Epoch 40/50
 1/32 [..............................] - ETA: 0s - loss: 0.0026 - sparse_categorical_accuracy: 1.0000
Epoch 40: saving model to training_2/cp-0040.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.0021 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.

In [11]:
os.listdir(checkpoint_dir)


['cp-0040.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001']

choose the latest one

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

To test, reset the model, and load the latest checkpoint.

In [13]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 [==============================] - 0s 649us/step - loss: 0.5048 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


## The files

The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
- One or more shards that contain your model's weights.
- An index file that indicates which weights are stored in which shard.

## Manually save weights

To save weights manually, use `tf.keras.Model.save_weights`. By default, `tf.keras`—and the `Model.save_weights` method in particular—uses the **TensorFlow Checkpoint** format with a `.ckpt` extension. To save in the **HDF5** format with a `.h5` extension, refer to the Save and load models guide.

In [14]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

#Restore
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 [==============================] - 0s 647us/step - loss: 0.5048 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


## Save the entire model
`Call tf.keras.Model.save` to save a model's architecture, weights, and training configuration in a single file/folder. This allows to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, we can resume training from exactly where we left off.
<br><br>
An entire model can be saved in two different file formats (`SavedModel` and `HDF5`). The TensorFlow `SavedModel` format is the default file format in TF2.x. However, models can be saved in `HDF5` format. More details on saving entire models in the two file formats is described below.
<br><br>
Saving a fully-functional model is very useful—we can load them in TensorFlow.js (Saved Model, HDF5) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (Saved Model, HDF5)
#### *Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the Saving custom objects section below.

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The SavedModel guide goes into detail about how to `serve/inspect` the SavedModel. 

In [15]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

#Save the entire model as a SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1248 - sparse_categorical_accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4147 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2923 - sparse_categorical_accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2054 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1490 - sparse_categorical_accuracy: 0.9700
INFO:tensorflow:Assets written to: saved_model/my_model/assets


2022-11-29 09:05:13.008584: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory

In [17]:
ls saved_model

my_model/


In [18]:
ls saved_model/my_model

assets/  keras_metadata.pb  saved_model.pb  variables/


### Reload a fresh keras model from the saved model

In [19]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss, acc = new_model.evaluate(test_images,test_labels, verbose=1)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 [==============================] - 0s 661us/step - loss: 0.4122 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.30%
(1000, 10)


## HDF5 format


In [28]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1564 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4191 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2818 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2093 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1481 - sparse_categorical_accuracy: 0.9700


In [29]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=1)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 [==============================] - 0s 623us/step - loss: 0.4164 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.30%


Keras saves models by inspecting their architectures. This technique saves everything:
- The weight values
- The model's training configuration (what is passed to the `.compile()` method)
- The optimizer and its state, if any (this enables to restart training where we left off)
